In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

guess_ages = np.zeros((2,3))

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()
            age_guess = guess_df.median()
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]
    dataset['Age'] = dataset['Age'].astype(int)

train_df['AgeBand'] = pd.cut(train_df['Age'], 5)

for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']

train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

freq_port = train_df.Embarked.dropna().mode()[0]

for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)

for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df.drop("Survived", axis=1), train_df["Survived"], test_size=0.3)
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
    elif train==False:
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))

In [ ]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)
print_score(clf, X_train, y_train, X_test, y_test, train=True)
print_score(clf, X_train, y_train, X_test, y_test, train=False) # Test

Train Result:

accuracy score: 0.8780

Classification Report: 
               precision    recall  f1-score   support

           0       0.88      0.93      0.90       384
           1       0.88      0.79      0.83       239

    accuracy                           0.88       623
   macro avg       0.88      0.86      0.87       623
weighted avg       0.88      0.88      0.88       623


Confusion Matrix: 
 [[358  26]
 [ 50 189]]

Average Accuracy: 	 0.8074
Accuracy SD: 		 0.0266
Test Result:

accuracy score: 0.8022

Classification Report: 
               precision    recall  f1-score   support

           0       0.81      0.88      0.85       165
           1       0.78      0.67      0.72       103

    accuracy                           0.80       268
   macro avg       0.80      0.78      0.78       268
weighted avg       0.80      0.80      0.80       268


Confusion Matrix: 
 [[146  19]
 [ 34  69]]



In [ ]:
bag_clf = BaggingClassifier(base_estimator=clf, n_estimators=1000,
                            bootstrap=True, n_jobs=-1,
                            random_state=42)
bag_clf.fit(X_train, y_train)
print_score(bag_clf, X_train, y_train, X_test, y_test, train=True)
print_score(bag_clf, X_train, y_train, X_test, y_test, train=False)

Train Result:

accuracy score: 0.8780

Classification Report: 
               precision    recall  f1-score   support

           0       0.89      0.91      0.90       384
           1       0.85      0.83      0.84       239

    accuracy                           0.88       623
   macro avg       0.87      0.87      0.87       623
weighted avg       0.88      0.88      0.88       623


Confusion Matrix: 
 [[349  35]
 [ 41 198]]

Average Accuracy: 	 0.8282
Accuracy SD: 		 0.0241
Test Result:

accuracy score: 0.7948

Classification Report: 
               precision    recall  f1-score   support

           0       0.81      0.87      0.84       165
           1       0.77      0.67      0.72       103

    accuracy                           0.79       268
   macro avg       0.79      0.77      0.78       268
weighted avg       0.79      0.79      0.79       268


Confusion Matrix: 
 [[144  21]
 [ 34  69]]



In [ ]:
bag_clf = BaggingClassifier(base_estimator=clf, n_estimators=1000,
                            bootstrap=True, oob_score=True,
                            n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_
print_score(bag_clf, X_train, y_train, X_test, y_test, train=True)
print_score(bag_clf, X_train, y_train, X_test, y_test, train=False)

Train Result:

accuracy score: 0.8780

Classification Report: 
               precision    recall  f1-score   support

           0       0.89      0.91      0.90       384
           1       0.85      0.83      0.84       239

    accuracy                           0.88       623
   macro avg       0.87      0.87      0.87       623
weighted avg       0.88      0.88      0.88       623


Confusion Matrix: 
 [[349  35]
 [ 41 198]]

Average Accuracy: 	 0.8282
Accuracy SD: 		 0.0241
Test Result:

accuracy score: 0.7948

Classification Report: 
               precision    recall  f1-score   support

           0       0.81      0.87      0.84       165
           1       0.77      0.67      0.72       103

    accuracy                           0.79       268
   macro avg       0.79      0.77      0.78       268
weighted avg       0.79      0.79      0.79       268


Confusion Matrix: 
 [[144  21]
 [ 34  69]]



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
    elif train==False:
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))

In [ ]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)


Train Result:

accuracy score: 0.8780

Classification Report: 
               precision    recall  f1-score   support

           0       0.89      0.91      0.90       384
           1       0.85      0.83      0.84       239

    accuracy                           0.88       623
   macro avg       0.87      0.87      0.87       623
weighted avg       0.88      0.88      0.88       623


Confusion Matrix: 
 [[349  35]
 [ 41 198]]

Average Accuracy: 	 0.8299
Accuracy SD: 		 0.0163
Test Result:

accuracy score: 0.8172

Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.89      0.86       165
           1       0.80      0.70      0.75       103

    accuracy                           0.82       268
   macro avg       0.81      0.79      0.80       268
weighted avg       0.82      0.82      0.81       268


Confusion Matrix: 
 [[147  18]
 [ 31  72]]



In [3]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier()
ada_clf.fit(X_train, y_train)
print_score(ada_clf, X_train, y_train, X_test, y_test, train=True)
print_score(ada_clf, X_train, y_train, X_test, y_test, train=False)
Y_pred = ada_clf.predict(test_df.drop('PassengerId',axis=1))
Y_pred
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submissions_ada.csv', index=False)

Train Result:

accuracy score: 0.8427

Classification Report: 
               precision    recall  f1-score   support

           0       0.87      0.89      0.88       401
           1       0.79      0.77      0.78       222

    accuracy                           0.84       623
   macro avg       0.83      0.83      0.83       623
weighted avg       0.84      0.84      0.84       623


Confusion Matrix: 
 [[355  46]
 [ 52 170]]

Average Accuracy: 	 0.8123
Accuracy SD: 		 0.0639
Test Result:

accuracy score: 0.7724

Classification Report: 
               precision    recall  f1-score   support

           0       0.76      0.85      0.81       148
           1       0.79      0.68      0.73       120

    accuracy                           0.77       268
   macro avg       0.78      0.76      0.77       268
weighted avg       0.77      0.77      0.77       268


Confusion Matrix: 
 [[126  22]
 [ 39  81]]



In [ ]:
from sklearn.ensemble import RandomForestClassifier
ada_clf = AdaBoostClassifier(RandomForestClassifier())
ada_clf.fit(X_train, y_train)
print_score(ada_clf, X_train, y_train, X_test, y_test, train=True)
print_score(ada_clf, X_train, y_train, X_test, y_test, train=False)
ada_clf = AdaBoostClassifier(base_estimator=RandomForestClassifier())
ada_clf.fit(X_train, y_train)
print_score(ada_clf, X_train, y_train, X_test, y_test, train=True)
print_score(ada_clf, X_train, y_train, X_test, y_test, train=False)
Y_pred = ada_clf.predict(test_df.drop('PassengerId',axis=1))
Y_pred
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submissions_ada_random.csv', index=False)

Train Result:

accuracy score: 0.8828

Classification Report: 
               precision    recall  f1-score   support

           0       0.88      0.95      0.91       401
           1       0.89      0.77      0.82       222

    accuracy                           0.88       623
   macro avg       0.88      0.86      0.87       623
weighted avg       0.88      0.88      0.88       623


Confusion Matrix: 
 [[379  22]
 [ 51 171]]

Average Accuracy: 	 0.8108
Accuracy SD: 		 0.0412
Test Result:

accuracy score: 0.7575

Classification Report: 
               precision    recall  f1-score   support

           0       0.73      0.88      0.80       148
           1       0.80      0.61      0.69       120

    accuracy                           0.76       268
   macro avg       0.77      0.74      0.75       268
weighted avg       0.76      0.76      0.75       268


Confusion Matrix: 
 [[130  18]
 [ 47  73]]

Train Result:

accuracy score: 0.8828

Classification Report: 
               p

In [ ]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(max_depth=5, n_estimators=10000, learning_rate=0.3,
                            n_jobs=-1)
xgb_clf.fit(X_train, y_train)
print_score(xgb_clf, X_train, y_train, X_test, y_test, train=True)
print_score(xgb_clf, X_train, y_train, X_test, y_test, train=False)
Y_pred = xgb_clf.predict(test_df.drop('PassengerId',axis=1))
Y_pred
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submissions_xgb.csv', index=False)

Train Result:

accuracy score: 0.8828

Classification Report: 
               precision    recall  f1-score   support

           0       0.88      0.95      0.91       401
           1       0.89      0.77      0.82       222

    accuracy                           0.88       623
   macro avg       0.88      0.86      0.87       623
weighted avg       0.88      0.88      0.88       623


Confusion Matrix: 
 [[379  22]
 [ 51 171]]

Average Accuracy: 	 0.8077
Accuracy SD: 		 0.0447
Test Result:

accuracy score: 0.7649

Classification Report: 
               precision    recall  f1-score   support

           0       0.74      0.89      0.81       148
           1       0.81      0.62      0.70       120

    accuracy                           0.76       268
   macro avg       0.78      0.75      0.75       268
weighted avg       0.77      0.76      0.76       268


Confusion Matrix: 
 [[131  17]
 [ 46  74]]

